COMMAND ----------

In [1]:
!pip install pdfservices-sdk==2.3.1
!pip install openpyxl
!pip install pandas

  Using cached pdfservices_sdk-2.3.1-py3-none-any.whl.metadata (2.3 kB)
  Using cached build-0.9.0-py3-none-any.whl.metadata (4.1 kB)
  Using cached certifi-2022.12.7-py3-none-any.whl.metadata (2.9 kB)
  Using cached cffi-1.15.1-cp311-cp311-macosx_11_0_arm64.whl.metadata (1.1 kB)
  Using cached cryptography-3.4.6-cp36-abi3-macosx_11_0_arm64.whl.metadata (5.1 kB)
  Using cached multipart-0.2.4-py3-none-any.whl.metadata (1.1 kB)
  Using cached packaging-21.3-py3-none-any.whl.metadata (15 kB)
  Using cached pep517-0.13.0-py3-none-any.whl.metadata (3.8 kB)
  Using cached polling-0.3.2-py3-none-any.whl
  Using cached polling2-0.5.0-py2.py3-none-any.whl.metadata (2.7 kB)
  Using cached pycparser-2.21-py2.py3-none-any.whl.metadata (1.1 kB)
  Using cached Pygments-2.14.0-py3-none-any.whl.metadata (1.6 kB)
  Using cached PyJWT-2.4.0-py3-none-any.whl.metadata (4.1 kB)
  Using cached pyparsing-3.0.9-py3-none-any.whl.metadata (4.2 kB)
  Using cached PyYAML-6.0-cp311-cp311-macosx_11_0_arm64.whl.met

In [1]:
from adobe.pdfservices.operation.auth.credentials import Credentials
from adobe.pdfservices.operation.exception.exceptions import (
    ServiceApiException,
    ServiceUsageException,
    SdkException,
)
from adobe.pdfservices.operation.execution_context import ExecutionContext
from adobe.pdfservices.operation.io.file_ref import FileRef
from adobe.pdfservices.operation.pdfops.extract_pdf_operation import ExtractPDFOperation
from adobe.pdfservices.operation.pdfops.options.extractpdf.extract_pdf_options import (
    ExtractPDFOptions,
)
from adobe.pdfservices.operation.pdfops.options.extractpdf.extract_element_type import (
    ExtractElementType,
)
from adobe.pdfservices.operation.pdfops.options.extractpdf.extract_renditions_element_type import (
    ExtractRenditionsElementType,
)
import os.path
import zipfile
import json
import pandas as pd
import re
import openpyxl
from datetime import datetime
import sys

sys.path.append("..")
from function.adobe import *

In [2]:
client_id = os.getenv("ADOBE_CLIENT_ID")
client_secret = os.getenv("ADOBE_CLIENT_SECRET")
client_id

In [ ]:
import glob
import os
import time

# Base directory for processed files
output_base_dir = "../data/processed/adobe_result"

# Get all PDF files in the raw data folder
all_pdf_files = glob.glob("../data/raw/Southern California Wedding Venues/*.pdf")

# Process each PDF file
for pdf_path in all_pdf_files:
    pdf_name = os.path.basename(pdf_path).replace(".pdf", "")
    # Check if output directory already exists
    output_pdf_dir = os.path.join(output_base_dir, pdf_name)
    if os.path.exists(output_pdf_dir):
        print(f"Skipping {pdf_name} as output directory already exists.")
        continue

    # Create output directories
    os.makedirs(output_pdf_dir)

    output_zip_path = os.path.join(output_pdf_dir, "sdk.zip")
    output_extract_folder = output_pdf_dir

    # Run adobe API
    max_retries = 3
    retry_delay = 5  # seconds
    success = False
    for attempt in range(max_retries):
        try:
            adobeLoader(
                pdf_path,
                output_zip_path,
                client_id=client_id,
                client_secret=client_secret,
            )
            print(f"Successfully processed {pdf_name}...")
            success = True
            break
        except Exception as e:
            if attempt < max_retries - 1:
                print(
                    f"Error processing {pdf_name}: {str(e)}. Try again on on attempt {attempt + 1}."
                )
                print(f"Retrying in {retry_delay} seconds...")
                time.sleep(retry_delay)
            else:
                print(f"Failed all attempts for {pdf_name}: {str(e)}")

Skipping Darlington House as output directory already exists.
Skipping Gainey_Brochure_Wedding2024 as output directory already exists.
Skipping Ridgemark Golf Club & Resort as output directory already exists.
Skipping orange county museum of art as output directory already exists.
Skipping Town & Gown USC as output directory already exists.
Skipping cypress sea cove as output directory already exists.
Skipping casita hollywood as output directory already exists.
Skipping Point San Luis Lighthouse Keepers as output directory already exists.
Skipping Villa and Vine as output directory already exists.
Skipping beverly hills presbyterian church as output directory already exists.
Skipping NOOR-Dinner Reception_BUFFET-merged as output directory already exists.
Skipping Vellano Estate Wedgewood Weddings as output directory already exists.
Skipping aquarium of the pacific as output directory already exists.
Skipping The Preserve Wedding Packages  as output directory already exists.
Skipping T

In [ ]:
import glob

output_base_zip_path = "../data/processed/adobe_result/"
all_files = glob.glob("../data/processed/adobe_result/*")
to_run_file = [os.path.join(output_base_zip_path, file) for file in all_files]
output_base_extract_folder = "../data/processed/adobe_extracted/"

for pdf_path in to_run_file:
    pdf_name = os.path.basename(pdf_path).replace(".pdf", "")
    output_zip_path = os.path.join(output_base_zip_path, pdf_name, f"sdk.zip")
    output_zipextract_folder = os.path.join(output_base_extract_folder, pdf_name)
    documents = extract_text_from_file_adobe(output_zip_path, output_zipextract_folder)